In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_style('whitegrid')
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from get_clean_data_BT import get_data, clean_data

In [ ]:
df = get_data(2016)

In [ ]:
df

In [ ]:
df.info()

<h1>Data Clean Up and Parsing</h1>

<h3>Convert object counts to numeric</h3>

In [ ]:
df_clean = clean_data(df)

In [ ]:
clean_list = ['part_cnt_PY', 'part_cnt_CY']#, 'sb_act_partcp_cnt','sb_term_partcp_cnt', 'sb_rtd_partcp_cnt']

for c in clean_list:
    df[c] = pd.to_numeric(df[c])

df['eir'] = np.where(df['eir']>100, df['eir']/100, df['eir'])

In [ ]:
df.info()

<h3>Add informational columns to parse data</h3>

In [ ]:
df['pay_related'] = df['type_pension_bnft_code'].str.contains('1A')
df['cash_bal'] = df['type_pension_bnft_code'].str.contains('1C')
df['frozen'] = df['type_pension_bnft_code'].str.contains('1I')
df['pbgc_takeover'] = df['type_pension_bnft_code'].str.contains('1H')
df['not_qual'] = df['type_pension_bnft_code'].str.contains('3B','3C')

In [ ]:
df.groupby('not_qual')['ein'].count()

<h3>Restrict analysis to: </h3>
<h5><ul><li>active participant count between 100 and 300,000
        <li>funding target non-zero
        <li>filing as tax qualified
    </ul>

In [ ]:
df.info()

In [ ]:
df = df[(df['fndng_tgt_2016'] > 0) & (df['fndng_tgt_2017'] > 0) &\
      (df['part_cnt_2017'] < 300000) & (df['part_cnt_2017'] > 100) & \
       (df['not_qual'] == False)]

In [ ]:
df.dropna(inplace=True)

In [ ]:
df.describe()

In [ ]:
X = df[['eir', 'part_cnt_2016', 'fndng_tgt_2016', 'tgt_nrml_cost_2016','distrib_drt_partcp_amt']]

In [ ]:
y = df['fndng_tgt_2017']

In [ ]:
X.describe()

<h1>Regression Tree</h1>

In [ ]:
from sklearn.tree import DecisionTreeRegressor

In [ ]:
dtr1 = DecisionTreeRegressor(max_depth=2)
dtr2 = DecisionTreeRegressor(max_depth=3)

In [ ]:
dtr1.fit(X,y)
dtr2.fit(X,y)

In [ ]:
y_1 = dtr1.predict(X)
y_2 = dtr2.predict(X)

In [ ]:
X.shape, y.shape, len(X)

In [ ]:
plt.figure(figsize=(20,8))
xx = np.linspace(0, len(X), num=len(X))
plt.scatter(xx, y, s=20, edgecolor="black", c="darkorange", label="data")
plt.plot(xx, y_1, color="cornflowerblue", label="max_depth=2", linewidth=2)
plt.plot(xx, y_2, color="yellowgreen", label="max_depth=3", linewidth=2)
plt.xlabel("data")
plt.ylabel("target")
plt.title("Decision Tree Regression")
plt.legend()
plt.show()

In [ ]:
dtr1.score(X,y)

In [ ]:
dtr2.score(X,y)